## Conventional Matrix Factorization & Random Predictor Baselines

In [1]:
# all aux imports
import pandas as pd
import numpy as np
from scipy.sparse.linalg import svds
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.sparse import csr_matrix
from pandas.api.types import CategoricalDtype
from sklearn.metrics import mean_squared_error
import pickle
import random
import glob
from pathlib import Path

# random seeds
random.seed(42)
np.random.seed(42)

# surplrise lib
from surprise import Dataset, Reader, SVD, NormalPredictor, BaselineOnly
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise.model_selection import GridSearchCV

# visualizaion
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

In [4]:
parent_path = r'D:\Datasets\amazon_reviews\gzips'
files = [Path(f) for f in glob.glob(parent_path  + r"\*.gz", recursive=False)]

for file in files:
    label = ' '.join(file.stem.split('_')[1:-1])
    print('Processing: ' + label)              
    chunk_num = 1 
    
    dfs = []
    for df_chunky in pd.read_json(str(file), lines=True, compression = 'gzip', chunksize=1000000):
        print('Chunk #%s' % chunk_num)
        chunk_num += 1
        dfs.append(df_chunky[['reviewerID', 'asin', 'overall']].copy())
        
    complete_df = pd.concat(dfs)
    print("Processing is completed, df shape %s" % complete_df.shape)
        

Processing: Books
Chunk #1
Chunk #2
Chunk #3
Chunk #4
Chunk #5
Chunk #6
Chunk #7
Chunk #8
Chunk #9


NameError: name 'df' is not defined

## Surprise Predictions

In [17]:
reader = Reader(rating_scale=(1, 5))

ds = Dataset.load_from_df(df[['reviewerID', 'asin', 'overall']], reader)
training_set, test_set = train_test_split(ds, random_state=42, test_size=0.3)

In [47]:
# We'll use the famous SVD algorithm.
algo = SVD()

# Run 5-fold cross-validation and print results
res = cross_validate(algo, ds, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0738  1.0741  1.0788  1.0838  1.0675  1.0756  0.0055  
MAE (testset)     0.8249  0.8247  0.8277  0.8329  0.8239  0.8268  0.0033  
Fit time          6.02    6.21    6.32    6.03    6.20    6.16    0.12    
Test time         0.19    0.23    0.23    0.19    0.72    0.31    0.20    


### Normal Predictor

In [19]:
normal_predictor = NormalPredictor()
normal_predictor.fit(training_set)
predictions = normal_predictor.test(test_set)

In [ ]:
accuracy.rmse(predictions)
accuracy.mae(predictions)

### Matrix Factorization

In [ ]:
algo = SVD(n_factors = 50, n_epochs = 1200, biased = False)
res = algo.fit(tr)

In [64]:
mean_squared_error(ratings, preds) ** 0.05

0.7987746310900994

In [75]:
preds_df_unmelt = pd.DataFrame(preds, columns = col_cat.categories, index = row_cat.categories)
preds_df_unmelt.index.name = 'reviewerID'
preds_df_unmelt.columns.name = 'asin'
preds_df_unmelt.head(2)

asin,0700099867,6050036071,7100027950,7293000936,8176503290,907843905X,9625990674,9861019731,9882155456,B000003SQQ,...,B00J128FPA,B00J226358,B00J6DLPLK,B00J9P3KBS,B00JM3R6M6,B00JQ8YH6A,B00JQHU9RC,B00JXW6GE0,B00KAI3KW2,B00KHECZXO
reviewerID,,,,,,,,,,,,,,,,,,,,,
A00263941WP7WCIL7AKWL,1.140781e-08,6.479067e-09,3.861577e-09,0.003669,2.945002e-07,2.556105e-09,1.978651e-08,4.670722e-08,9.912429e-07,7.284264e-10,...,5.929648e-09,9.978265e-09,9.835851e-09,1.212900e-08,1.390002e-07,8.875097e-03,2.160884e-08,5.116919e-09,2.111927e-08,1.664742e-09
A005481137I9SCAWEF7ON,5.843863e-08,3.413003e-08,2.884808e-03,0.000002,5.650788e-06,4.459515e-06,8.162578e-06,7.296751e-07,4.127624e-05,3.806345e-09,...,4.574513e-06,3.954006e-04,7.341937e-06,6.336393e-08,6.583915e-08,1.064613e-07,1.691314e-02,3.088186e-08,1.218786e-07,2.553060e-06


In [76]:
preds_df = preds_df_unmelt.reset_index().melt('reviewerID', var_name='asin')

In [78]:
preds_df.head(2)

,reviewerID,asin,value
0,A00263941WP7WCIL7AKWL,0700099867,1.140781e-08
1,A005481137I9SCAWEF7ON,0700099867,5.843863e-08


In [100]:
%reset Out

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
Flushing output cache (18 entries)


In [81]:
df_merged = df_test.merge(preds_df, on=['reviewerID', 'asin'])

In [83]:
df_merged.shape

(69363, 10)

In [85]:
df_test.shape

(69534, 9)

In [91]:
df_merged.head(20)

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime,value
0,B0047TLIBU,"[31, 43]",5,"Been playing this for two days now, and I am v...","03 10, 2011",A17M1HL6U2GS7M,Storylover,It feels like hanging out with old friends aga...,1299715200,5.993661e-04
1,B0053BCP40,"[0, 0]",4,"I really enjoyed playing this game, although i...","02 27, 2014",A1FGU7F9UJ264N,Kelly Randall,Fun game,1393459200,2.271945e-07
2,B000WCQWR6,"[0, 0]",3,This was kind of a gamble and was looking for ...,"07 7, 2013",A3R7M2EP1EYNVT,Thomas B. Hileman Jr.,Shoot em up,1373155200,3.205947e-02
3,B00498T500,"[4, 4]",4,***Updated 7/7/2011 - update marked below***I ...,"04 10, 2011",A369KP0JV77JYT,Jon,"Fun for a group, don't pay too much for it",1302393600,6.258070e-03
4,B00005BIG7,"[3, 3]",5,It looks like we've come a long way from past ...,"07 13, 2001",ABIKTKAWOMY8,Chance Farley,One of the best fps's to ever hit a console,994982400,5.010131e-03
5,B002CYWJWO,"[7, 8]",4,"Tellurye, here with another review brought to ...","10 30, 2009",A2SYA663GK3EEJ,"R. Stevens ""Fedora Technologies""","Does the Saw universe justice, but controls ma...",1256860800,2.050861e-04
6,B001V9PTVE,"[2, 2]",4,Fallout 2 is worth the price alone... but if y...,"09 6, 2009",A39AUQ3IP3C8KI,J. Nicholson,Worth it (even if it was just FO2),1252195200,6.256847e-09
7,B00DBDPOZ4,"[1, 1]",5,"As I'm not a fan of AA batteries, this accesso...","12 4, 2013",A22N09ZP9D77DM,D. Alvarado,Great upgrade over the Xbox 360 version.,1386115200,1.186235e-01
8,B0009A4EV2,"[4, 4]",5,"Next to the Final Fantasy series, the Dragon Q...","03 22, 2007",AJKWF4W7QD4NS,"N. Durham ""Big Evil""","One of the best games, ever, for the PS2",1174521600,4.627793e-01
9,B00002EQAP,"[0, 0]",5,"This game is so great, not just for the crazy ...","08 20, 2013",A1T4HGVX32QIYC,Will Ramirez,Great Game,1376956800,4.788158e-09


In [101]:
mean_squared_error(df_merged.overall, df_merged.value) ** 0.5

4.226522013061119

In [105]:
sum((df_merged.overall - df_merged.value) < 3)

17259

In [109]:
idx = ((df_merged.overall - df_merged.value < 1) & (df_merged.overall > 1))

In [110]:
df_merged[idx].shape

(23, 10)